In [1]:
pip install tensorflow tensorflow-datasets scikit-learn opencv-python


In [29]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the dataset (train and test)
(ds_train, ds_test), ds_info = tfds.load('cats_vs_dogs',
                                         split=['train[:80%]', 'train[80%:]'],
                                         with_info=True, as_supervised=True)

# Preprocessing: Resize and convert to grayscale
IMG_SIZE = 128

def preprocess(image, label):
    # Resize image
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    # Convert to grayscale
    image = tf.image.rgb_to_grayscale(image)
    # Normalize the image to [0, 1]
    image = image / 255.0
    return image, label

# Apply preprocessing
ds_train = ds_train.map(preprocess)
ds_test = ds_test.map(preprocess)

# Convert to NumPy arrays
def ds_to_numpy(ds, num_samples):
    images = []
    labels = []
    for img, lbl in ds.take(num_samples):
        images.append(img.numpy().flatten())  # Flatten the image
        labels.append(lbl.numpy())
    return np.array(images), np.array(labels)

# Convert the dataset to numpy arrays for training
NUM_TRAIN_SAMPLES = 2000  # You can adjust the size
NUM_TEST_SAMPLES = 500

X_train, y_train = ds_to_numpy(ds_train, NUM_TRAIN_SAMPLES)
X_test, y_test = ds_to_numpy(ds_test, NUM_TEST_SAMPLES)

print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")




Training data shape: (2000, 16384), Test data shape: (500, 16384)


In [30]:
# Encode labels (cat = 0, dog = 1)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Create the SVM classifier
svm = SVC(kernel='rbf', verbose=True)

# Train the model
svm.fit(X_train, y_train_encoded)

# Make predictions on the test set
y_pred = svm.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


[LibSVM]Accuracy: 63.20%


In [33]:
def preprocess_image(image_path, img_size=128):
    # Read the image file
    img = tf.io.read_file(image_path)
    # Decode the image to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to the size used for training (IMG_SIZE x IMG_SIZE)
    img = tf.image.resize(img, (img_size, img_size))
    # Convert to grayscale to match the preprocessing used during training
    img = tf.image.rgb_to_grayscale(img)
    # Normalize the image (scaling pixel values to the range [0, 1])
    img = img / 255.0
    # Flatten the image into a 1D array
    img = img.numpy().flatten().reshape(1, -1)  # Reshape to match SVM input format
    return img

In [34]:
# Test on an external image
image_path = '/content/dog.jpg'  # Provide the path to your test image here
preprocessed_img = preprocess_image(image_path)

# Make a prediction
prediction = svm.predict(preprocessed_img)

# Decode the prediction (cat = 0, dog = 1)
label = le.inverse_transform(prediction)
print(f"The predicted label is: {label[0]}")

The predicted label is: 1
